In [4]:
%load_ext autoreload
%autoreload 2

# Add the path to the parent directory to sys
import sys
sys.path.append('../attribution')

import transformers
import torch
import inseq
import datasets

from constants import ModelNames
from model_utils import load_pretrained

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
model, tokenizer = load_pretrained(ModelNames.QwenInstruct)